In [4]:
from csv import DictReader
from tabulate import tabulate
from random import randrange
from IPython.display import clear_output

### Helper Functions

In [196]:
class Villager:
    def __init__(self, id, product):
        """ 
        Creates a villager based on units consumed per person per week.
        Inventory will be initialized with negative values as initially a villager will be in deficit of those values
        """
        self.id = id
        self.product = product
        self.inventory = {}
        for row in data:
            self.inventory[row['product']] = -1 * float(row['units_consumed_pppw'])

    def __str__(self):
        return f'{str(self.id)}:{self.product}'

    def __repr__(self):
        return self.__str__()

    def add_product(self, product, units):
        """ updates villager's inventory by giving units of a product to them """
        self.inventory[product] += units

    def remove_product(self, product, units):
        """ updates villager's inventory by taking units of a product from them """
        self.inventory[product] -= units
        
    def set_product(self, product, units):
        """ updates villager's inventory by setting product to given units """
        self.inventory[product] = units

    def has_product(self, product, min_units):
        """ checks if the villager has sufficient units of product """
        return self.inventory[product] >= min_units

    def can_sell(self):
        """ returns list of product surplus in inventory that villager wants to sell """
        surplus = []
        for product in self.inventory:
            if self.inventory[product] > 0:
                surplus.append(product)
        return surplus

    def can_buy(self):
        """ returns list of product deficit in inventory that villager wants to buy """
        deficit = []
        for product in self.inventory:
            if self.inventory[product] < 0:
                deficit.append(product)
        return deficit

    def get_product_to_exchange(self):
        """ villager decides product_to_exchange based on maximum product he can take by giving product_to_exchange """ 
        product_to_exchange = None
        hours_to_exchange = None
        for sellable_product in self.can_sell():
            if hours_to_exchange is None or hours_to_exchange < units_produced_per_hour[sellable_product] * self.inventory[sellable_product]:
                hours_to_exchange = units_produced_per_hour[sellable_product] * self.inventory[sellable_product]
                product_to_exchange = sellable_product
        return product_to_exchange
    

    def has_product_to_sell(self):
        return len(self.can_sell()) > 0

    def has_product_to_buy(self):
        return len(self.can_buy()) > 0


def load_csv(file):
    """ loads csv file to dict """
    with open(file, 'r', encoding='utf-8') as f:
        dict_reader = DictReader(f)
        return list(dict_reader)

def get_producer_distribution(population, data):
    """ defines number of producers for a product based on population and producers_percent """
    producer_distribution = {}
    for row in data:
        producer_distribution[row['product']] = round((population * float(row['producers_percent']))/100)
    return producer_distribution

def get_units_produced_per_hour(data):
    """ returns units_produced_per_hour from data """
    units_produced_per_hour = {}
    for row in data:
        units_produced_per_hour[row['product']] = float(row['units_produced_per_hour'])
    return units_produced_per_hour

def create_village(producer_distribution, units_produced_per_hour, work_hours):
    """ 
    Creates a village by adding villagers based on producer_distribution.
    Initially each villager will get units equivalent of production_rate times work_hours of their product
    """
    village = []
    id = 0
    for product in producer_distribution:
        for producers in range(producer_distribution[product]):
            villager = Villager(id, product)
            villager.add_product(product, units_produced_per_hour[product] * work_hours)
            village.append(villager)
            id += 1
    return village

def print_village(village):
    """ Prints inventory of all the villagers """
    village_data = []
    for villager in village:
        formatted_villager = vars(villager) | villager.inventory
        formatted_villager.pop('inventory', None)
        village_data.append(formatted_villager)

    print(tabulate(village_data, headers='keys', tablefmt='fancy_grid'))

def get_exchange_rate(units_produced_per_hour):
    """ creates two dimensional table of exchange rates between the produces """
    exchange_rate = {}
    for product1 in units_produced_per_hour:
        product1_exchange_rates = {}
        for product2 in units_produced_per_hour:
            product1_exchange_rates[product2] = units_produced_per_hour[product2]/units_produced_per_hour[product1]
        exchange_rate[product1] = product1_exchange_rates
    return exchange_rate

def print_exchange_rate(exchange_rate):
    """ Prints exchange rate table for 1 unit of product on y-axis get number of units of product on x-axis """
    exchange_rate_table = []
    for product in exchange_rate:
        exchange_rate_table.append({'↓ 1 unit \\ gets →':product} | exchange_rate[product])  
    print(tabulate(exchange_rate_table, headers='keys', tablefmt='fancy_grid'))

def pick_random_from(collection):
    return list(collection)[randrange(len(collection))]

class Market:
    def __init__(self, village, exchange_rate):
        """ Initializes the market by identifying villagers willing to sell or buy products """
        self.market_is_open = True
        self.buyers = {}
        self.sellers = {}
        self.transactions = []
        self.exchange_rate = exchange_rate
        
        buyer_products = set()
        seller_products = set()

        for villager in village:
            if villager.has_product_to_buy():
                for product in villager.can_buy():
                    self.add_buyer(product, villager)
                    buyer_products.add(product)
                
            if villager.has_product_to_sell():
                for product in villager.can_sell():
                    self.add_seller(product, villager)
                    seller_products.add(product)

        self.products = buyer_products.intersection(seller_products)

    @staticmethod
    def add_villager(trader_dict, product, villager):
        """ Adds a villager to the product list in trader_dict, if product is not present the creates a new list """
        if product in trader_dict and villager not in trader_dict[product]:
            trader_dict[product].append(villager)
        else:
            trader_dict[product] = [villager]

    def add_buyer(self, product, villager):
        Market.add_villager(self.buyers, product, villager)

    def add_seller(self, product, villager):
        Market.add_villager(self.sellers, product, villager)

    @staticmethod
    def exchange(buyer, product, product_units, seller, product_to_exchange, product_to_exchange_units):
        """ Performs exchange of product of given units between traders """
        seller.remove_product(product, product_units)
        buyer.remove_product(product_to_exchange, product_to_exchange_units)
        seller.add_product(product_to_exchange, product_to_exchange_units)
        buyer.add_product(product, product_units)

    @staticmethod
    def trade(buyer, product, seller, product_to_exchange, exchange_rate):
        """ Perform trade of product, product_to_exchange between buyer, seller based on minimum giving and taking capacity of each """
        buyer_giving_capacity = buyer.inventory[product_to_exchange] * exchange_rate[product_to_exchange][product]
        buyer_taking_capacity = -1 * buyer.inventory[product]
        seller_giving_capacity = seller.inventory[product]
        seller_taking_capacity = -1 * seller.inventory[product_to_exchange] * exchange_rate[product_to_exchange][product]
        if seller_taking_capacity > 0:
            product_units = min(buyer_giving_capacity, buyer_taking_capacity, seller_giving_capacity, seller_taking_capacity)
            trade_type = 'direct'
        else:
            product_units = min(buyer_giving_capacity, buyer_taking_capacity, seller_giving_capacity)
            trade_type = 'transitive'
        product_to_exchange_units = product_units * exchange_rate[product][product_to_exchange]
        print(f'trade: bgc:{buyer_giving_capacity}, sgc:{seller_giving_capacity}, pu:{product_units}, peu: {product_to_exchange_units}')
        Market.exchange(buyer, product, product_units, seller, product_to_exchange, product_to_exchange_units)
        return {
            'buyer': buyer, 'product': product, 'product_units': product_units, 
            'seller': seller, 'product_to_exchange': product_to_exchange, 'product_to_exchange_units': product_to_exchange_units,
            'trade_type': trade_type, 'buyer_giving_capacity': buyer_giving_capacity/exchange_rate[product][product_to_exchange], 'buyer_taking_capacity': buyer_taking_capacity,
            'seller_giving_capacity': seller_giving_capacity, 'seller_taking_capacity': seller_taking_capacity/exchange_rate[product][product_to_exchange],
            'spi': seller.inventory[product_to_exchange]
            }
    
    def get_random_seller(self, product, product_to_exchange):
        """ finds a random seller who can give product and take product_to_exchange, if unavailable then just a random seller who can give product"""
        direct_sellers = set(self.sellers[product]).intersection(set(self.buyers[product_to_exchange]))
        print(f'{product} - direct_sellers:{direct_sellers}, all_sellers:{self.sellers[product]}')
        return pick_random_from(direct_sellers) if len(direct_sellers) > 0 else pick_random_from(self.sellers[product])

    def execute(self):
        """ 
        Simulates the market by picking random product and then performs trading between random buyer and
        random seller. When a villager has no deficit or no surplus then the villager is removed from the buyers
        or sellers list for the product. When a product has no buyers or sellers, then the product is removed
        from the products list. Market remains open until products list is not empty.
        """
        while self.market_is_open:
            product = pick_random_from(self.products)
            buyer = pick_random_from(self.buyers[product])
            if buyer.has_product_to_sell():
                product_to_exchange = buyer.get_product_to_exchange()
                seller = self.get_random_seller(product, product_to_exchange)
                #print(f'product={product}, buyer={buyer.product}, product_to_exchange={product_to_exchange}, seller={seller.product}')
                print(f'buyer={buyer} product_to_exchange={product_to_exchange} buyer can sell {buyer.can_sell()}:{str(buyer.inventory[product_to_exchange])}, all sellers={self.sellers[product_to_exchange]}')
                transaction = Market.trade(buyer, product, seller, product_to_exchange, exchange_rate)
                self.transactions.append(transaction)
                if transaction['trade_type'] == 'transitive':
                    self.add_seller(product_to_exchange, seller)

                print(f'transaction: {transaction}')
                print(f'buyers[{product}]: ' + str(self.buyers[product]))
                print(f'sellers[{product}]: ' + str(self.sellers[product]))
                print(f'buyers[{product_to_exchange}]: ' + str(self.buyers[product_to_exchange]))
                print(f'sellers[{product_to_exchange}]: ' + str(self.sellers[product_to_exchange]))
                print('buyer: ' + str(buyer))
                print('seller: ' + str(seller))

                if not product in buyer.can_buy():
                    self.buyers[product].remove(buyer)
                if not product_to_exchange in buyer.can_sell():
                    print(f'buyer={buyer} product_to_exchange={product_to_exchange} buyer can sell {buyer.can_sell()}:{str(buyer.inventory[product_to_exchange])}, all sellers={self.sellers[product_to_exchange]}')
                    self.sellers[product_to_exchange].remove(buyer)
                if not product in seller.can_sell():
                    self.sellers[product].remove(seller)
                if not product_to_exchange in seller.can_buy() and transaction['trade_type'] == 'direct':
                    print(f'seller={seller} product_to_exchange={product_to_exchange} seller can buy {seller.can_buy()}:{seller.inventory[product_to_exchange]}, all buyers={self.buyers[product_to_exchange]}')
                    self.buyers[product_to_exchange].remove(seller)

            else:
                print(f'Error: Buyer {str(buyer)} is broke')
                self.buyers[product].remove(buyer)
            
            if len(self.sellers[product]) == 0:
                self.sellers.pop(product, None)
                self.products.remove(product)

            if len(self.buyers[product]) == 0:
                self.buyers.pop(product, None)
                self.products.remove(product)

            if len(self.sellers[product_to_exchange]) == 0:
                self.sellers.pop(product_to_exchange, None)
                self.products.remove(product_to_exchange)

            if len(self.buyers[product_to_exchange]) == 0:
                self.buyers.pop(product_to_exchange, None)
                self.products.remove(product_to_exchange)

            self.market_is_open = len(self.products) > 0
            clear_output(wait=True)

    def print_transactions(self):
        print(f'{len(self.transactions)} transactions')
        print(tabulate(self.transactions, headers='keys', tablefmt='fancy_grid'))

            



### Initialise

In [197]:
data = load_csv('data.csv')
population = 100
work_hours = 40

producer_distribution = get_producer_distribution(population, data)
units_produced_per_hour = get_units_produced_per_hour(data)
exchange_rate = get_exchange_rate(units_produced_per_hour)
village = create_village(producer_distribution, units_produced_per_hour, work_hours)

In [198]:
#print_exchange_rate(exchange_rate)

In [199]:
#print_village(village)

### Execute

In [200]:
market = Market(village, exchange_rate)
market.execute()

meat - direct_sellers:set(), all_sellers:[49:meat, 50:meat, 51:meat, 52:meat, 4:wheat, 85:tourism, 87:health]
buyer=76:cloth product_to_exchange=milk buyer can sell ['milk', 'constructions_repairs', 'law']:1.1102230246251565e-16, all sellers=[58:milk, 59:milk, 60:milk, 61:milk]
trade: bgc:2.2204460492503132e-17, sgc:0.07999999999999605, pu:2.2204460492503132e-17, peu: 1.1102230246251565e-16
transaction: {'buyer': 76:cloth, 'product': 'meat', 'product_units': 2.2204460492503132e-17, 'seller': 50:meat, 'product_to_exchange': 'milk', 'product_to_exchange_units': 1.1102230246251565e-16, 'trade_type': 'transitive', 'buyer_giving_capacity': 4.440892098500626e-18, 'buyer_taking_capacity': 3.497202527569243e-15, 'seller_giving_capacity': 0.07999999999999605, 'seller_taking_capacity': -0.0, 'spi': 1.1102230246251565e-16}
buyers[meat]: [1:wheat, 5:rice, 27:vegetables, 37:fruits, 47:chicken, 54:eggs, 76:cloth, 84:transportation, 88:constructions_repairs, 92:council, 96:law, 99:police]
sellers[mea

ValueError: list.remove(x): x not in list

In [201]:
# Scratchpad
market.print_transactions()

1415 transactions
╒══════════════════════════╤═══════════════════════╤═════════════════╤══════════════════════════╤═══════════════════════╤═════════════════════════════╤══════════════╤═════════════════════════╤═════════════════════════╤══════════════════════════╤══════════════════════════╤══════════════╕
│ buyer                    │ product               │   product_units │ seller                   │ product_to_exchange   │   product_to_exchange_units │ trade_type   │   buyer_giving_capacity │   buyer_taking_capacity │   seller_giving_capacity │   seller_taking_capacity │          spi │
╞══════════════════════════╪═══════════════════════╪═════════════════╪══════════════════════════╪═══════════════════════╪═════════════════════════════╪══════════════╪═════════════════════════╪═════════════════════════╪══════════════════════════╪══════════════════════════╪══════════════╡
│ 16:vegetables            │ transportation        │     2.4         │ 81:transportation        │ vegetables          

In [ ]:
#x = 4
#print(f"{x} wheat = {x * exchange_rate['wheat']['vegetables']} vegetables")
a = 1/3
b = a/6
print(a + a + a)

1.0
